# Guide to Using Python to Accompany Business Analytics by Jaggia et al.
## Chapter 7
##### Prepared by David G. Dobolyi at the University of Notre Dame

### Table of Contents

- [Example 7.8](#Example-7.8)
- [Examples 7.10 and 7.11](#Examples-7.10-and-7.11)
- [Example 7.14](#Example-7.14)

### Example 7.8

Revisit the regression models in Example 7.7 and determine which model is best suited for making predictions.

#### Solution

From Table 7.14 in the text, Model 4 has the highest computer-generated $R^2$ value of 0.8613. However, this does not mean that Model 4 is necessarily the best, since $R^2$ is based on Rent for Models 1 and 2 and on ln(Rent) for models 3 and 4. Therefore, while we can infer that Model 2 is superior to Model 1 (0.8482 > 0.8092) and Model 4 is superior to Model 3 (0.8613 > 0.8095), we cannot directly compare Models 2 and 4 based on the computer-generated $R^2$. For a valid comparison, we compute $R^2$ for Model 4 from scratch; that is, $R^2$ is based on Rent, even though it uses ln(Rent) for estimation.

For Model 4, we first compute $\widehat{Rent} = exp(b_0 + b_1Beds + b_2Baths + b_3ln(Sqft) + s_e^2/2)$ for the given sample values of the predictor variables. For example, for the first sample observation, with Beds = 1, Baths = 1, and Sqft = 500, the predicted is computed as

$$\widehat{Rent} = exp(3.3808 + 0.1246(1) + 0.0254(1) + 0.4742 \times ln(500) + 0.1262^2/2) = \$656)$$

The predicted rent for other sample observations is calculated similarly. Excel, R, and Python are quite useful in performing the calculations for the predicted values for Rent, as well as for generating a value of $R^2$ for Model 4 based on Rent rather than ln(Rent).

**a.** Import the ***AnnArbor*** data into a Pandas DataFrame (table) and label it myData.

In [1]:
import pandas as pd

myData = pd.read_excel('jaggia_ba_1e_ch07_Data_Files.xlsx', sheet_name = 'AnnArbor')

myData.head()

Rent  Beds  Baths  Sqft
0   645     1    1.0   500
1   675     1    1.0   648
2   760     1    1.0   700
3   800     1    1.0   903
4   820     1    1.0   817

**b.** As shown in Chapter 6, we use the **ols** function in statsmodels to create a regression model and store this model in a variable named Model4. We also use the NumPy **log** function for the natural log transformation of the variables Rent and Sqft. Finally, we use the statsmodels **fit** and **summary** functions to fit and summarize Model4:

In [2]:
import statsmodels.formula.api as smf
import numpy as np

Model4 = smf.ols(formula = 'np.log(Rent) ~ Beds + Baths + np.log(Sqft)', data = myData).fit()

Model4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           np.log(Rent)   R-squared:                       0.861
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     74.51
Date:                Wed, 15 Jul 2020   Prob (F-statistic):           1.65e-15
Time:                        17:08:15   Log-Likelihood:                 28.145
No. Observations:                  40   AIC:                            -48.29
Df Residuals:                      36   BIC:                            -41.53
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        3.3808      0.772      4.379      0.000       1.815       4.946
Beds             0.1246      0.045      2.758      0.009       0.033       0.216
Baths            0.0254      0.039      0.659      0.514      -0.053       0.104
np.log(Sqft)     0.4742      0.124      3.821      0.001       0.222       0.726
==============================================================================
Omnibus:                        3.492   Durbin-Watson:                   2.044
Prob(Omnibus):                  0.174   Jarque-Bera (JB):                2.914
Skew:                          -0.094   Prob(JB):                        0.233
Kurtosis:                       4.309   Cond. No.                         303.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

**c.** We then generate predicted values for Model4, or $\widehat{ln(Rent)}$, using the statsmodels **predict** function. Within the function, we indicate the name of the model and store the result in a variable named Pred_lnRent. Enter:

In [3]:
Pred_lnRent = Model4.predict()

**d.** Next, we want to calculate the predicted values for Rent: $\widehat{Rent} = exp(\widehat{ln(Rent)} + s_e^2/2)$. We label $\widehat{Rent}$ as Pred_Rent. To obtain the standard error of the estimate and store it in a variable named SE, we use the **scale** property to extract it from Model 4. Next, we use the NumPy **exp** function to calculate the exponential based on our formula. Enter:

In [4]:
SE = np.sqrt(Model4.scale)

Pred_Rent = np.exp(Pred_lnRent + (SE ** 2)/2)

Note that the symbol **\*\*** is used to raise a value to a power in Python.

**e.** Finally, we use NumPy's **corrcoef** function to calculate the correlation between Rent and $\widehat{Rent}$. We square this value to find the coefficient of determination, $R^2$. Enter:

In [5]:
np.corrcoef(myData.Rent, Pred_Rent) ** 2

array([[1.        , 0.75541879],
       [0.75541879, 1.        ]])

And Python returns a correlation matrix, in which either of the two identical off-diagonal values shows our desired result (i.e., 0.75541879). We again conclude that Model 2, with an $R^2$ of 0.8482, is better suited than Model 4 for making predictions.

### Examples 7.10 and 7.11

Let's revisit Example 7.9.

- Estimate an interpret the logistic regression model for the loan approval outcome $y$ based on the applicant's percentage of down payment $x_1$ and the income-to-loan ratio $x_2$.  
- For an application with a 30\% income-to-loan ratio, predict loan approval probabilities with down payments of 20\% and 30\%.  
- Compare the predicted probabilities based on the estimated logistic regression model with those from the estimated linear probability model in Example 7.9.

In addition, for the mortgage approval example, compare the accuracy rates of the estimated linear probability model (LPM) with the estimated logistic regression model.

#### Solution

Similar to the text, we show how to solve these examples in Python and recreate various tables and calculations shown in the text.

**a.** Import the ***Mortgage*** data into a Pandas DataFrame (table) and label it myData.

In [6]:
import pandas as pd

myData = pd.read_excel('jaggia_ba_1e_ch07_Data_Files.xlsx', sheet_name = 'Mortgage')

myData.head()

y     x1     x2
0  1  16.35  49.94
1  1  34.43  56.16
2  1  39.19  36.89
3  1  23.58  56.88
4  0  29.92  27.05

**b.** We use the statsmodels **glm** function to construct a logistic regression model and label it as Logistic_Model. Within the function, we first specify $y$ as a function of $x1$ and $x2$ and then use the **families.Binomial** function to define the type of model. Finally, we specify the data source using the *data* argument. We use the **fit** and **summary** functions to fit and summarize the model, respectively.

In [7]:
from statsmodels.api import families
import statsmodels.formula.api as smf

Logistic_Model = smf.glm('y ~ x1 + x2', family = families.Binomial(), data = myData).fit()

Logistic_Model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                      y   No. Observations:                   30
Model:                            GLM   Df Residuals:                       27
Model Family:                Binomial   Df Model:                            2
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -10.917
Date:                Wed, 15 Jul 2020   Deviance:                       21.835
Time:                        17:08:15   Pearson chi2:                     30.2
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -9.3671      3.196     -2.931      0.003     -15.631      -3.103
x1             0.1349      0.064      2.107      0.035       0.009       0.260
x2             0.1782      0.065      2.758      0.006       0.052       0.305
==============================================================================
"""

The regression summary output is identical to that shown in Figure 7.14 and Table 7.18 in the text.

As is the case of the linear probability model, both predictor variables exert a positive and statistically significant influence on loan approval at a 5\% level, given positive estimated coefficients and $p$-values of 0.035 and 0.006, respectively. (In maximum likelihood estimation, the significance tests are valid only with large samples. Consequently, we conduct the $z$ test, in place of the usual $t$ test, to evaluate the statistical signficance of a coefficient.)

**c.** We use the **predict** function to compute the predicted probabilities for the given sample values, labeled as Pred. Enter:

In [8]:
Pred = Logistic_Model.predict()

**d.** We use the NumPy **round** function to construct the binary predicted values, labeled as Binary. Due to how rounding is defined in this particular Python function, if a predicted probability is greater than 0.50, then its corresponding binary value is 1; if a predicted probability is less than or equal to 0.5, then its corresponding binary value is 0. Enter:

In [9]:
import numpy as np

Binary = np.round(Pred)

**e.** Both $y$ and Binary now contain 0 and 1 values. We want to find the proportion of the values in the sample for which $y$ equals Binary. Recall from Chapter 2 that the double equal sign (==) is used to compare the values of $y$ and Binary. If the two values are the same, the operator returns 1, and 0 otherwise. We use the NumPy **mean** function to compute the proportion of correctly classified observations; we multiply by 100 to get a percentage. Enter:

In [10]:
np.mean(myData.y == Binary) * 100

86.66666666666667

This result is identical to that shown for the logistic regression model in Example 7.11 in the text.

**f.** To make predictions for applicants, we can use the **predict** function by supplying relevant predictor values in a Pandas DataFrame and then round the result, as discussed in the Chapter 6 examples. Enter:

In [11]:
Logistic_Predictions = Logistic_Model.predict(pd.DataFrame({
    'x1': [20, 30], 
    'x2': [30, 30]
    }))

round(Logistic_Predictions, 4)

0    0.2104
1    0.5066
dtype: float64

The predicted loan approval probability with $x_1$ = 20 and $x_2$ = 30 is 0.2104, and the predicted probability for $x_1$ = 30 and $x_2$ = 30 is 0.5066. These values are identical to that shown in the text, albeit with slight differences due to rounding.

**g.** To calculate the predicted probabilities for the logistic regression model shown in Table 7.19 in the text, we can use **predict** again:

In [12]:
More_Logistic_Predictions = Logistic_Model.predict(pd.DataFrame({
    'x1': [ 5, 20, 30, 50, 60], 
    'x2': 30
    }))

round(More_Logistic_Predictions, 4)

0    0.0340
1    0.2104
2    0.5066
3    0.9385
4    0.9833
dtype: float64

These results are identical to those shown in Table 7.19 in the text, although again their are minor differences due to rounding. Note that we set $x2$ to a constant value of 30 when defining the Pandas DataFrame in the code since the same value was used for every (varying) observation of $x1$.

**h.** To calculate the predicted probabilities for the linear probability model from Example 7.9 with Python, enter:

In [13]:
Linear_Model = smf.ols('y ~ x1 + x2', data = myData).fit()

Linear_Model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.506
Model:                            OLS   Adj. R-squared:                  0.469
Method:                 Least Squares   F-statistic:                     13.82
Date:                Wed, 15 Jul 2020   Prob (F-statistic):           7.37e-05
Time:                        17:08:16   Log-Likelihood:                -10.931
No. Observations:                  30   AIC:                             27.86
Df Residuals:                      27   BIC:                             32.07
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.8682      0.281     -3.089      0.005      -1.445      -0.291
x1             0.0188      0.007      2.694      0.012       0.004       0.033
x2             0.0258      0.006      4.107      0.000       0.013       0.039
==============================================================================
Omnibus:                        1.526   Durbin-Watson:                   2.217
Prob(Omnibus):                  0.466   Jarque-Bera (JB):                0.712
Skew:                           0.357   Prob(JB):                        0.700
Kurtosis:                       3.247   Cond. No.                         194.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The coefficients shown in the summary are identical to those presented in Examples 7.9 and 7.11 in the text. Next, we calculate the predictions:

In [14]:
Linear_Predictions = Linear_Model.predict(pd.DataFrame({
    'x1': [5, 20, 30, 50, 60], 
    'x2': 30
    }))

round(Linear_Predictions, 4)

0    0.0010
1    0.2836
2    0.4720
3    0.8488
4    1.0372
dtype: float64

The results are again consistent with Table 7.19 in the text, although again there are differences due to rounding and the precision of the coefficients used for the calculations.

**i.** To calculate the accurate rate for the linear probability model, enter:

In [15]:
np.mean(myData.y == np.round(Linear_Model.predict())) * 100

83.33333333333334

This result is identical to the accuracy rate shown in the text in Example 7.11.

**j.** Finally, we can place the combined results in a Pandas DataFrame to facilitate comparison. Enter:

In [16]:
Prediction_Table = pd.DataFrame({
    'x1': [5, 20, 30, 50, 60], 
    'x2': 30,
    'Linear Probability': Linear_Predictions,
    'Logistic': More_Logistic_Predictions
    })

round(Prediction_Table, 4)

x1  x2  Linear Probability  Logistic
0   5  30              0.0010    0.0340
1  20  30              0.2836    0.2104
2  30  30              0.4720    0.5066
3  50  30              0.8488    0.9385
4  60  30              1.0372    0.9833

As discussed earlier, with the linear probability model, the predicted probabilities can be negative or greater than one. The probabilities based on the logistic regression model stay between zero and one for all possible values of the predictor variables. Therefore, whenever possible, it is preferable to use the logistic regression model over the linear probability model in binary choice models.

**k.** We can also recreate table 7.20 using Pandas and NumPy. Specifically, we use the Pandas **assign** function add the relevant columns for the model predictions to the existing myData variables. The predictions are calculated using functions we have used previously in this example, including **predict** and NumPy's **round**. Moreover, we convert the binary predictions to integers using NumPy's **astype** so that they are formated as 0s and 1s rather than `False` and `True` in boolean, respectively. Enter:

In [17]:
Predicted_Result_Table = myData.assign(
    Prediction_LPM = Linear_Model.predict(),
    Prediction_Logistic = Logistic_Model.predict(),
    Binary_Prediction_LPM = np.round(Linear_Model.predict()).astype(int),
    Binary_Prediction_Logistic = np.round(Logistic_Model.predict()).astype(int))

round(Predicted_Result_Table, 4)

y     x1     x2  Prediction_LPM  Prediction_Logistic  \
0   1  16.35  49.94          0.7299               0.8506   
1   1  34.43  56.16          1.2312               0.9950   
2   1  39.19  36.89          0.8231               0.9237   
3   1  23.58  56.88          1.0453               0.9811   
4   0  29.92  27.05          0.3943               0.3752   
5   1  25.26  44.38          0.7541               0.8755   
6   1  36.51  48.98          1.0849               0.9864   
7   1  11.70  55.55          0.7872               0.8920   
8   0  32.21  31.28          0.5467               0.6348   
9   1  28.74  35.63          0.5937               0.7027   
10  1  18.28  39.50          0.4966               0.5346   
11  0  10.12  31.39          0.1333               0.0826   
12  0  10.39  29.47          0.0888               0.0622   
13  0  21.46  29.34          0.2940               0.2239   
14  1  33.56  40.37          0.8069               0.9133   
15  1  37.91  22.92          0.4381               0.4580   
16  1  31.81  47.56          0.9597               0.9677   
17  0  25.88  44.58          0.7710               0.8879   
18  1  38.40  47.85          1.0913               0.9871   
19  0  26.62  25.50          0.2920               0.2259   
20  0  14.36  21.87         -0.0327               0.0284   
21  1  22.22  20.79          0.0875               0.0651   
22  1  32.10  51.56          1.0684               0.9845   
23  0  11.75  32.96          0.2046               0.1292   
24  1  10.32  48.59          0.5814               0.6649   
25  0  11.43  34.78          0.2456               0.1643   
26  0  12.58  33.27          0.2282               0.1492   
27  0  27.53  25.63          0.3126               0.2525   
28  1  36.71  37.05          0.7805               0.8992   
29  0  17.85  26.86          0.1620               0.1023   

    Binary_Prediction_LPM  Binary_Prediction_Logistic  
0                       1                           1  
1                       1                           1  
2                       1                           1  
3                       1                           1  
4                       0                           0  
5                       1                           1  
6                       1                           1  
7                       1                           1  
8                       1                           1  
9                       1                           1  
10                      0                           1  
11                      0                           0  
12                      0                           0  
13                      0                           0  
14                      1                           1  
15                      0                           0  
16                      1                           1  
17                      1                           1  
18                      1                           1  
19                      0                           0  
20                      0                           0  
21                      0                           0  
22                      1                           1  
23                      0                           0  
24                      1                           1  
25                      0                           0  
26                      0                           0  
27                      0                           0  
28                      1                           1  
29                      0                           0

Outside of minor differences to rounding and calculation precision, these results closely align with Table 7.20 in the text.

### Example 7.14

Peter Derby works as a cyber security analyst at a private equity firm. His colleagues at the firm have been inundated by a large number of spam e-mails. Peter has been asked to implement a spam detection system on the company's e-mail server. He analyzes a sample of 500 spam and legitimate emails with the following relevant variables: spam (1 if spam, 0 otherwise), the number of recipients, the number of hyperlinks, and the number of characters in the message. A portion of the data is shown in the accompanying table.

| Table 7.25 Spam Data ($n$ = 500) |
| --- |

| Record | Spam | Recipients | Hyperlinks | Characters |
| --- | --- | --- | --- | --- |
| 1 | 0 | 19 | 1 | 47 |
| 2 | 0 | 15 | 1 | 58 |
| ... | ... | ... | ... | ... |
| 500 | 1 | 13 | 2 | 32 |

- Use the holdout method to compare the accuracy rates of two competing logistic models for spam, using the first 375 observations for training and the remaining 125 observations for validation. Model 1 uses Recipients, Hyperlinks, and Characters as predictor variables, whereas Model 2 drops the predictor variable(s) found to be statistically insignificant in Model 1.

#### Solution

Python is easy to use for partitioning the data, estimating models with the training set, and deriving the necessary cross-validation measures. As mentioned earlier, we generally use random draws for partioning the sample data into the training and validation sets. Here, for replicating purposes, we will continue to use the latter part of the sample data for the validation set. We use the ***Spam*** data to replicate the results in Example 7.14 for Model 1; results for Model 2 can be derived similarly.

**a.** Import the ***Spam*** data into a Pandas DataFrame (table) and label it myData.

In [18]:
import pandas as pd

myData = pd.read_excel('jaggia_ba_1e_ch07_Data_Files.xlsx', sheet_name = 'Spam')

myData.head()

Record  Spam  Recipients  Hyperlinks  Characters
0       1     0          19           1          47
1       2     0          15           1          58
2       3     1          13          11          88
3       4     1          17          11          68
4       5     0          15           1          87

**b.** We partition the sample data into training and validation sets, labeled TData and VData, respectively. Enter:

In [19]:
TData = myData[0:375]
VData = myData[375:]

Note that for TData we use the slice `0:375` since the first row has index 0 and the stop value of 375 is not inclusive (i.e., we will get row indicies 0 through 374, which are the first 375 rows). For VData, `375:` returns all rows from 375 (inclusive) to the end of myData (i.e., 375 - 499, or the last 125 rows).

**c.** We use the training set, TData, to estimate Model 1 using statsmodels as shown in [Example 7.10](#Example-7.10) part b above, making sure to set the *data* argument accordingly. Enter:

In [20]:
from statsmodels.api import families
import statsmodels.formula.api as smf

Model1 = smf.glm('Spam ~ Recipients + Hyperlinks + Characters', family = families.Binomial(), data = TData).fit()

Model1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:                   Spam   No. Observations:                  375
Model:                            GLM   Df Residuals:                      371
Model Family:                Binomial   Df Model:                            3
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -160.59
Date:                Wed, 15 Jul 2020   Deviance:                       321.17
Time:                        17:08:16   Pearson chi2:                     361.
No. Iterations:                     5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -5.1778      0.864     -5.992      0.000      -6.871      -3.484
Recipients     0.1765      0.045      3.927      0.000       0.088       0.265
Hyperlinks     0.5473      0.054     10.210      0.000       0.442       0.652
Characters    -0.0104      0.006     -1.787      0.074      -0.022       0.001
==============================================================================
"""

**d.** We use the estimates to make predictions for VData and then convert them into a binary prediction. Finally, we compute the accuracy rate in the validation set. Enter:

In [21]:
import numpy as np

Pred1 = Model1.predict(VData)

Binary1 = round(Pred1)

np.mean(VData.Spam == Binary1) * 100

68.0

This is the same as derived for Model 1 in Example 7.14.